# Python tutorial: funkcijų panaudojimas praktikoje (pardavimai, PVM, marketingo biudžetas)

## Tikslas

- Parodyti, kaip kurti ir naudoti funkcijas sprendžiant praktines analitines užduotis.
- Derinti built-in funkcijas (`sum`, `min`, `max`, `round`, `sorted`, `len`, `enumerate`) su sukurtomis funkcijomis (`def`).
- Pateikti paprastus, aiškius verslo pavyzdžius su sąrašais, žodynais, tuple ir ciklais.
- Paminėti dažniausias klaidas ir kaip jų išvengti.

## 1) Kodėl funkcijos praverčia analitikoje

Funkcijos leidžia:
- vieną kartą aprašyti skaičiavimo logiką ir ją panaudoti daug kartų,
- sumažinti kodo kartojimą,
- aiškiau atskirti „skaičiavimo taisyklę“ nuo „duomenų“.

## 2) Built-in funkcijos ir sukurtos funkcijos

- Built-in funkcijos yra jau paruoštos Python kalboje, pavyzdžiui `sum()`, `round()`, `len()`.
- Sukurtos funkcijos (`def`) aprašo konkrečią verslo taisyklę, pavyzdžiui „pritaikyti PVM“, „suskaičiuoti užsakymo sumą“.

Žemiau pateikiamas trumpas built-in funkcijų pavyzdys.

In [16]:
values = [100, 80, 40]
print("sum:", sum(values))
print("min:", min(values))
print("max:", max(values))
print("len:", len(values))
print("round:", round(12.3456, 2))
print("sorted:", sorted(values, reverse=False))

sum: 220
min: 40
max: 100
len: 3
round: 12.35
sorted: [40, 80, 100]


## 3) Duomenų pavyzdys: pardavimų eilutės

Paprastam modeliavimui naudojamas sąrašas su žodynais.
Kiekviena eilutė turi:
- kanalą,
- produkto pavadinimą,
- kiekį,
- vieneto kainą,
- nuolaidą (dalimi, pvz. 0.10 reiškia 10%).

In [17]:
sales_lines = [
    {"channel": "online",  "product": "Milk",   "qty": 2, "unit_price": 1.50, "discount": 0.00},
    {"channel": "online",  "product": "Bread",  "qty": 1, "unit_price": 0.99, "discount": 0.10},
    {"channel": "direct",  "product": "Eggs",   "qty": 3, "unit_price": 0.80, "discount": 0.00},
    {"channel": "partner", "product": "Butter", "qty": 1, "unit_price": 2.20, "discount": 0.05},
]
print(sales_lines)

[{'channel': 'online', 'product': 'Milk', 'qty': 2, 'unit_price': 1.5, 'discount': 0.0}, {'channel': 'online', 'product': 'Bread', 'qty': 1, 'unit_price': 0.99, 'discount': 0.1}, {'channel': 'direct', 'product': 'Eggs', 'qty': 3, 'unit_price': 0.8, 'discount': 0.0}, {'channel': 'partner', 'product': 'Butter', 'qty': 1, 'unit_price': 2.2, 'discount': 0.05}]


## 4) Funkcija: užsakymo eilutės suma

Užsakymo eilutės suma dažnai skaičiuojama taip:
`qty * unit_price * (1 - discount)`

Svarbu:
- `discount` laikyti dalimi (0.10 = 10%).
- Galutinę sumą apvalinti `round()` built-in funkcija.

In [20]:
def line_total(qty, unit_price, discount=0.0):
    return round(qty * unit_price * (1 - discount), 2)

print(line_total(2, 1.50))
print(line_total(1, 0.99, 0.50))

3.0
0.49


### Dažna klaida: nuolaidą įvesti procentais (pvz., 10 vietoje 0.10)

Jei nuolaida įrašoma kaip 10, gaunamas neigiamas rezultatas.
Žemiau pateikiamas pavyzdys ir paprasta apsauga.

In [ ]:
print("Neteisinga nuolaida 10:", line_total(1, 100, 10))  # 10 reiškia 1000%

def normalize_discount(discount):
    if discount > 1:
        return discount / 100
    return discount

print("Normalizuota 10%:", line_total(1, 100, normalize_discount(10)))

## 5) Funkcija: PVM pritaikymas

PVM skaičiavimui dažniausiai taikoma taisyklė:
`amount * (1 + vat_rate)`

PVM tarifą patogu turėti kaip default argumentą.

In [ ]:
def add_vat(amount, vat_rate=0.21):
    return round(amount * (1 + vat_rate), 2)

print(add_vat(121))
print(add_vat(100, vat_rate=0.09))

## 6) Pardavimų sumavimas ciklu: bendra suma be PVM ir su PVM

- Ciklas panaudojamas eilučių apdorojimui.
- Built-in `round()` naudojamas rezultatui suformatuoti.

In [ ]:
net_total = 0.0
for row in sales_lines:
    net_total += line_total(row["qty"], row["unit_price"], row["discount"])

gross_total = add_vat(net_total, vat_rate=0.21)

print("Suma be PVM:", round(net_total, 2))
print("Suma su PVM:", gross_total)

## 7) Funkcija: suskaičiuoti pardavimus pagal kanalą

Verslo praktikoje dažnai reikia suvestinių pagal dimensiją (pvz., kanalą).
Tai panašu į „GROUP BY“ logiką SQL kalboje.

In [ ]:
def revenue_by_channel(lines):
    result = {}
    for row in lines:
        ch = row.get("channel")
        if not isinstance(ch, str) or ch.strip() == "":
            ch = "unknown"
        ch = ch.strip().lower()

        amount = line_total(row["qty"], row["unit_price"], row["discount"])
        result[ch] = result.get(ch, 0.0) + amount

    for k in result:
        result[k] = round(result[k], 2)
    return result

rev_ch = revenue_by_channel(sales_lines)
rev_ch

### Built-in funkcijų derinimas: rasti geriausią kanalą pagal pajamas

- `dict.items()` grąžina poras (raktas, reikšmė).
- `max()` su `key=...` randa didžiausią reikšmę.

In [ ]:
best = max(rev_ch.items(), key=lambda kv: kv[1])
print("Geriausias kanalas:", best)

## 8) Funkcija: ataskaitos eilutės su numeracija

`enumerate()` (built-in) patogus ataskaitos eilučių numeravimui.

In [ ]:
def build_report(lines):
    report = []
    for i, row in enumerate(lines, start=1):
        amount = line_total(row["qty"], row["unit_price"], row["discount"])
        report.append(f"{i}. {row['channel']} | {row['product']} | qty={row['qty']} | kaina_po_nuolaidos={amount}")
    return report

print("\n".join(build_report(sales_lines)))

## 9) Marketingo biudžeto planavimas

Paprastas planavimo modelis:
- turėti biudžetą,
- paskirstyti pagal kanalų svorius (weights),
- įvertinti tikslinius KPI, pvz., CPA (kaina už įsigijimą) arba konversijas.

In [ ]:
channel_weights = {"online": 0.500002, "direct": 0.300007, "partner": 0.200001}
channel_weights

### Funkcija: patikrinti, ar svorių suma lygi 1.0

Dažna klaida planavime yra svorių suma, kuri nėra 1.0.
Maža paklaida gali būti toleruojama.

In [ ]:
def weights_sum_ok(weights, expected=1.0, tolerance=0.0001):
    s = sum(weights.values())
    return abs(s - expected) <= tolerance, round(s, 4)

ok, s = weights_sum_ok(channel_weights)
print("Suma OK:", ok, "| suma:", s)

### Funkcija: paskirstyti biudžetą pagal svorius

- `total_budget` pateikiamas eurais.
- `weights` yra žodynas su kanalais ir jų dalimis.
- Grąžinamas žodynas su priskirtu biudžetu.

In [ ]:
def allocate_budget(total_budget, weights):
    ok, s = weights_sum_ok(weights)
    if not ok:
        return None, f"Netinkama svorių suma: {s}. Turi būti 1.0."

    allocation = {}
    for channel, w in weights.items():
        allocation[channel] = round(total_budget * w, 2)

    diff = round(total_budget - sum(allocation.values()), 2)
    if diff != 0:
        top_channel = max(weights.items(), key=lambda kv: kv[1])[0]
        allocation[top_channel] = round(allocation[top_channel] + diff, 2)

    return allocation, None

alloc, err = allocate_budget(1000, channel_weights)
print(alloc, err)

### Funkcija: prognozuoti konversijas pagal CPA

Paprasta logika:
- jei žinomas CPA, konversijų skaičius = `budget / cpa`,
- dalyba iš nulio ir trūkstami raktai turi būti apsaugoti.

In [ ]:
cpa = {"online": 120, "direct": 20, "partner": 25}

def forecast_conversions(allocation, cpa_map):
    result = {}
    for channel, budget in allocation.items():
        cpa_value = cpa_map.get(channel)
        if cpa_value is None or cpa_value == 0:
            result[channel] = None
        else:
            result[channel] = int(budget / cpa_value)
    return result

conversions = forecast_conversions(alloc, cpa)
conversions

## 10) Dažniausios klaidos ir jų priežastys

- Pamirštas `return` ir gautas `None` vietoje rezultato.
- Maišomi tipai (pvz., tekstas vietoje skaičiaus) ir gaunamas `TypeError`.
- Netinkamai interpretuojama nuolaida (10 vietoje 0.10).
- Per anksti apvalinama kiekviename žingsnyje ir prarandamas tikslumas.
- Nesuvaldoma dalyba iš nulio arba trūkstami raktai žodyne (`KeyError` vietoje `.get()`).
- Kanalų svoriai nesudaro 1.0, biudžetas paskirstomas neteisingai.

In [ ]:
prices = {"Milk": 1.50}

try:
    print(prices["Coffee"])
except KeyError as e:
    print("Klaida:", e)

print("Saugiau su get:", prices.get("Coffee", 0.0))

## Santrauka

- Sukurtos funkcijos leidžia įgyvendinti verslo skaičiavimo taisykles (eilutės suma, PVM, suvestinės, biudžeto paskirstymas).
- Built-in funkcijos padeda su agregacijomis ir tvarkymu (`sum`, `max`, `round`, `len`, `enumerate`).
- Praktikoje svarbu validuoti įvestis (nuolaidos formatas, svorių suma, dalyba iš nulio) ir naudoti `.get()` dirbant su žodynais.